In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import re

In [3]:
ruta = "Mexico/"
archivo = 'LIVEPOOLC1.xlsx'
ruta_archivo = ruta + archivo
anio_actual = datetime.now().year
anio_empieza_analisis = anio_actual - 10

In [4]:
# Reemplaza esto con la ruta a tu archivo
ruta_archivo = ruta_archivo
df = pd.read_excel(ruta_archivo, header=19)
# Filas a eliminar (ajustar índices para que sean 0-based)
filas_a_eliminar = [1, 16, 53, 86, 150, 191, 212]
# Eliminar las filas
df = df.drop(filas_a_eliminar)
df.columns = df.iloc[0]
if '\t' in df.columns:
    df = df.drop(columns=['\t'])
df = df[1:].reset_index(drop=True)

In [5]:
archivo_market="Datos históricos S&P_BMV IRT"
data_ipc = pd.read_csv(archivo_market + '.csv')
data_ipc['Fecha'] = pd.to_datetime(data_ipc['Fecha'], format='%d.%m.%Y')
data_ipc['Cierre'] = data_ipc['Cierre'].str.replace(',', '').astype(float)
data_fecha_ipc = data_ipc.sort_values('Fecha')
fecha_actual_ipc = data_fecha_ipc['Fecha'].max()
fecha_inicio_ipc = fecha_actual_ipc - pd.DateOffset(years=5)
datos_x_anos_ipc = data_fecha_ipc[data_fecha_ipc['Fecha'] >= fecha_inicio_ipc]
precio_inicial_ipc = datos_x_anos_ipc.iloc[0]['Cierre']
precio_final_ipc = datos_x_anos_ipc.iloc[-1]['Cierre']
n = 5  # número de años
crecimiento_ipc = ((precio_final_ipc / precio_inicial_ipc) ** (1 / n)) - 1

crecimiento_ipc_porcentaje = crecimiento_ipc * 100

print(f"5-Year Growth IPC: {crecimiento_ipc_porcentaje:.2f}%")

5-Year Growth IPC: 5.31%


In [6]:
cetes = 0.0835
cetes_porcentaje = cetes * 100
print(f"CETES: {cetes_porcentaje:.2f}%")

CETES: 8.35%


In [7]:
archivo_inflacion_mexico = "INFLACION_mexico"
data_inflacion_mexico = pd.read_excel(archivo_inflacion_mexico + '.xlsx')
data_inflacion_mexico  = data_inflacion_mexico.iloc[-20:-1]/100
data_inflacion_mexico_promedio = data_inflacion_mexico.mean().values[1]
data_inflacion_mexico_promedio

0.04466842105263158

In [8]:
archivo_stock = "Datos históricos LIVEPOLC1"

In [9]:
# Cargar datos históricos de la acción y del índice
rf_rate = cetes / 52  # Tasa libre de riesgo semanal 

# Leer archivos CSV
data_stock_beta = pd.read_csv(archivo_stock + '.csv')
data_market_beta = pd.read_csv(archivo_market + '.csv')

# Convertir las fechas y ordenar
data_stock_beta['Fecha'] = pd.to_datetime(data_stock_beta['Fecha'], format='%d.%m.%Y')
data_market_beta['Fecha'] = pd.to_datetime(data_market_beta['Fecha'], format='%d.%m.%Y')

data_stock_beta = data_stock_beta.sort_values('Fecha')
data_market_beta = data_market_beta.sort_values('Fecha')

# Convertir la columna 'Cierre' a tipo numérico, eliminando caracteres no válidos
data_stock_beta['Cierre'] = data_stock_beta['Cierre'].astype(str).str.replace(',', '').astype(float)
data_market_beta['Cierre'] = data_market_beta['Cierre'].astype(str).str.replace(',', '').astype(float)

# Verificar valores nulos y eliminarlos
data_stock_beta = data_stock_beta.dropna(subset=['Cierre'])
data_market_beta = data_market_beta.dropna(subset=['Cierre'])

# Calcular rendimientos semanales
data_stock_beta['Rend_Stock'] = data_stock_beta['Cierre'].pct_change()
data_market_beta['Rend_Market'] = data_market_beta['Cierre'].pct_change()

# Eliminar valores nulos generados por pct_change
data_stock_beta = data_stock_beta.dropna()
data_market_beta = data_market_beta.dropna()

# Combinar ambos DataFrames por fecha
merged_data = pd.merge(data_stock_beta[['Fecha', 'Rend_Stock']], 
                       data_market_beta[['Fecha', 'Rend_Market']], on='Fecha')

# Calcular rendimientos en exceso (Rendimiento - Tasa libre de riesgo semanal)
merged_data['Excess_Stock'] = merged_data['Rend_Stock'] - rf_rate
merged_data['Excess_Market'] = merged_data['Rend_Market'] - rf_rate

# Calcular covarianza y varianza
cov = np.cov(merged_data['Excess_Stock'], merged_data['Excess_Market'])[0, 1]
var_market = np.var(merged_data['Excess_Market'])

# Calcular Beta
beta = cov / var_market

print(f"Beta de la acción: {beta:.2f}")

Beta de la acción: 0.90


In [10]:
# Obtengo los DF para datos anulaes y trimestrales
indice_ttm = df.columns.get_loc('TTM/current')
# Crear una copia del DataFrame para datos anuales
df_anual = df.iloc[:, :indice_ttm + 1]
# Crear una copia del DataFrame para datos trimestrales
df_trimestral = df.iloc[:, indice_ttm + 1:]
df_trimestral.insert(0, 'Fiscal Period', df['Fiscal Period'])


In [11]:
columna_anterior = df_anual.columns[indice_ttm - 1]
anio_anterior = int(columna_anterior[-4:])
# Identificar las columnas que contienen un año en su nombre
pattern = re.compile(r'\d{4}')
columnas_con_anio = [col for col in df_anual.columns if pattern.search(col)]

# Filtrar las columnas que están dentro del rango de los últimos 10 años desde anio_anterior
anio_limite = anio_anterior - 10
columnas_filtradas = [col for col in columnas_con_anio if int(
    pattern.search(col).group()) >= anio_limite]

# Asegurarse de incluir 'Fiscal Period' y 'TTM/current' si están presentes
columnas_especiales = ['Fiscal Period', 'TTM/current']
columnas_filtradas = [
    col for col in df_anual.columns if col in columnas_especiales] + columnas_filtradas

# Crear un nuevo DataFrame con las columnas filtradas
df_anual_filtrado = df_anual[columnas_filtradas]

# Mover la columna 'TTM/current' al final
if 'TTM/current' in df_anual_filtrado.columns:
    columnas_ordenadas = [
        col for col in df_anual_filtrado.columns if col != 'TTM/current'] + ['TTM/current']
    df_anual_filtrado = df_anual_filtrado[columnas_ordenadas]

df_anual_filtrado = df_anual_filtrado.fillna(0)

df_anual_filtrado = df_anual_filtrado.loc[:, ~df_anual_filtrado.columns.str.contains("2020")]

In [12]:
pattern = re.compile(r'\d{4}')
columnas_con_anio_trimestral = [
    col for col in df_trimestral.columns if pattern.search(col)]

# Obtener el año de la última columna con año en su nombre
ultima_columna = columnas_con_anio_trimestral[-1]
anio_ultima_columna = int(pattern.search(ultima_columna).group())

# Filtrar las columnas que están dentro del rango de los últimos 10 años desde anio_ultima_columna
anio_limite_trimestral = anio_ultima_columna - 10
columnas_filtradas_trimestral = [col for col in columnas_con_anio_trimestral if int(
    pattern.search(col).group()) >= anio_limite_trimestral]

# Asegurarse de incluir 'Fiscal Period' si está presente
columnas_especiales_trimestral = ['Fiscal Period']

# Crear un nuevo DataFrame con las columnas filtradas
df_trimestral_filtrado = df_trimestral[columnas_especiales_trimestral +
                                       columnas_filtradas_trimestral]

df_trimestral_filtrado = df_trimestral_filtrado.fillna(0)

In [13]:
df_trimestral_filtrado = df_trimestral_filtrado.loc[:, ~df_trimestral_filtrado.columns.str.contains("2020")]


In [14]:
# Crear DataFrame vacío con las columnas anuales
razones_financieras = pd.DataFrame(
    columns=df_anual_filtrado.columns.difference(['Fiscal Period']))


def agregar_razon_financiera(df_anual_filtrado, fila_numerador, nombre_nueva_fila, *filas_denominadoras, como_porcentaje=True):

    global razones_financieras

    # Obtener la fila numerador
    fila_num = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == fila_numerador].iloc[0, 1:]

    # Obtener y sumar las filas denominadoras
    fila_den = sum(df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == fila].iloc[0, 1:].values for fila in filas_denominadoras)

    # Calcular la razón financiera, manejando la división por cero
    razon_financiera = []
    for num, den in zip(fila_num.values, fila_den):
        if den != 0:
            razon = num / den
            if como_porcentaje:
                razon *= 100
            razon_financiera.append(f"{razon:.2f}")
        else:
            razon_financiera.append("no deuda")

    # Agregar el resultado al DataFrame
    razones_financieras.loc[nombre_nueva_fila] = razon_financiera

def agregar_razon_financiera_numerador(df_anual_filtrado, nombre_nueva_fila, fila_denominador, fila_numerador_1, fila_numerador_2, como_porcentaje=True):

    global razones_financieras

    # Obtener las filas numeradoras
    fila_num_1 = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == fila_numerador_1].iloc[0, 1:]
    fila_num_2 = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == fila_numerador_2].iloc[0, 1:]

    # Obtener la fila denominador
    fila_den = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == fila_denominador].iloc[0, 1:]

    # Restar las filas numeradoras
    resta_numeradores = fila_num_1.values - fila_num_2.values

    # Calcular la razón financiera, manejando la división por cero
    razon_financiera = []
    for num, den in zip(resta_numeradores, fila_den.values):
        if den != 0:
            razon = num / den
            if como_porcentaje:
                razon *= 100
            razon_financiera.append(f"{razon:.2f}")
        else:
            razon_financiera.append("no deuda")

    # Agregar el resultado al DataFrame razones_financieras
    razones_financieras.loc[nombre_nueva_fila] = razon_financiera

def agregar_fila(df_anual_filtrado, fila_nombre, nombre_nueva_fila):

    global razones_financieras

    # Obtener la fila relevante
    fila = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == fila_nombre].iloc[0, 1:]

    # Agregar el resultado al DataFrame
    razones_financieras.loc[nombre_nueva_fila] = fila.values

def agregar_crecimiento_anual(fila_nombre, nombre_nueva_fila):

    global razones_financieras

    # Obtener la fila relevante
    fila = razones_financieras.loc[fila_nombre].astype(float)

    # Calcular el crecimiento año a año (YoY)
    crecimiento_anual = fila.pct_change() * 100

    # Reemplazar NaN por 0
    crecimiento_anual = crecimiento_anual.fillna(0)

    # Formatear el resultado a dos decimales
    crecimiento_anual_formateado = [f"{x:.2f}" for x in crecimiento_anual]

    # Agregar el resultado al DataFrame
    razones_financieras.loc[nombre_nueva_fila] = crecimiento_anual_formateado

def dividir_filas_razones_financieras(fila_numerador, fila_denominador, nombre_nueva_fila, como_porcentaje=True):
    global razones_financieras

    # Obtener las filas relevantes para el cálculo
    fila_num = razones_financieras.loc[fila_numerador].astype(float)
    fila_den = razones_financieras.loc[fila_denominador].astype(float)

    # Calcular la razón financiera
    razon_financiera = fila_num.values / fila_den.values

    # Convertir a porcentaje si es necesario
    if como_porcentaje:
        razon_financiera *= 100

    # Reemplazar NaN por 0
    razon_financiera = pd.Series(razon_financiera).fillna(0).values

    # Formatear el resultado a dos decimales
    razon_financiera_formateada = [f"{x:.2f}" for x in razon_financiera]

    # Agregar el resultado al DataFrame
    razones_financieras.loc[nombre_nueva_fila] = razon_financiera_formateada



In [15]:
df_anual_filtrado = df_anual_filtrado.drop(columns=["TTM/current"])

In [16]:
razones_financieras = razones_financieras.drop(columns=["TTM/current"])

In [17]:
# Llamadas a la función con el nuevo orden de parámetros
agregar_razon_financiera(df_anual_filtrado, 'Depreciation, Depletion and Amortization', 'Amortizacion(%)', 'Gross Profit', como_porcentaje=True)
agregar_razon_financiera(df_anual_filtrado, 'Net Income', 'Beneficio Neto(%)', 'Revenue', como_porcentaje=True)
agregar_fila(df_anual_filtrado, 'Net Income', 'Beneficio Neto')
agregar_razon_financiera(df_anual_filtrado, 'Gross Profit', 'Margen Bruto(%)', 'Revenue', como_porcentaje=True)
agregar_razon_financiera(df_anual_filtrado, 'Operating Income', 'Margen Operativo(%)', 'Revenue', como_porcentaje=True)
agregar_razon_financiera(df_anual_filtrado, 'Net Income', 'EPS', 'Shares Outstanding (Diluted Average)', como_porcentaje=False)
agregar_razon_financiera(df_anual_filtrado, 'Revenue', 'Revenue Per Share', 'Shares Outstanding (Diluted Average)', como_porcentaje=False)
agregar_fila(df_anual_filtrado, 'Shares Outstanding (Diluted Average)', '# de Acciones')
agregar_fila(df_anual_filtrado, 'Revenue', 'Ventas')
agregar_crecimiento_anual('Beneficio Neto', 'Crecimiento Beneficio Neto')
agregar_crecimiento_anual('Ventas', 'Crecimiento Ventas')
agregar_fila(df_anual_filtrado, 'Cost of Goods Sold', 'Costo de Ventas')
agregar_crecimiento_anual('Costo de Ventas', 'Crecimiento Costo de Ventas')
agregar_fila(df_anual_filtrado, 'Total Inventories', 'Inventarios')
agregar_crecimiento_anual('Inventarios', 'Crecimiento Inventarios')
agregar_razon_financiera(df_anual_filtrado, '  Accounts Receivable', 'Cuentas por cobrar/ventas', 'Revenue', como_porcentaje=True)
agregar_fila(df_anual_filtrado, 'Dividends per Share', 'Dividendos por Accion')
agregar_fila(df_anual_filtrado, 'Free Cash Flow', 'Flujo de Caja Libre')
dividir_filas_razones_financieras('Flujo de Caja Libre', 'Ventas', 'FCF/ventas',True)
dividir_filas_razones_financieras('Flujo de Caja Libre','# de Acciones','Free cash flow per share',True)
agregar_razon_financiera(df_anual_filtrado,'Free Cash Flow','FCF/DEBT','Short-Term Debt & Capital Lease Obligation','Long-Term Debt & Capital Lease Obligation',como_porcentaje=False)
agregar_razon_financiera(df_anual_filtrado,'Free Cash Flow','FCF/DEBT corto plazo','Short-Term Debt & Capital Lease Obligation',como_porcentaje=False)
dividir_filas_razones_financieras('Dividendos por Accion',"EPS",'Dividend Payout Ratio',True)
agregar_razon_financiera(df_anual_filtrado,'Net Income','Net Income vs Deuda Largo Plazo','Long-Term Debt & Capital Lease Obligation',como_porcentaje=False)
agregar_razon_financiera(df_anual_filtrado,'Total Liabilities','Pasivo/Fondos Propios','Total Equity',como_porcentaje=False)
agregar_razon_financiera(df_anual_filtrado,'Total Current Assets','Ratio de Solvencia','Total Current Liabilities',como_porcentaje=False)
agregar_razon_financiera(df_anual_filtrado,'Long-Term Debt & Capital Lease Obligation','Deuda/Capital','Total Equity',como_porcentaje=False)
agregar_razon_financiera(df_anual_filtrado,'  Interest Expense','Gasto financiero','Operating Income',como_porcentaje=True) ##Interes Expense viene negativo, tenerlo en cuenta ya que si queda positivo seria malo ya que el operating income es tambien negativo
agregar_razon_financiera(df_anual_filtrado,'Total Inventories','Inventory to current Assetis','Total Current Assets',como_porcentaje=True)
agregar_fila(df_anual_filtrado,'Cash Conversion Cycle','Net Trading Cycle')
agregar_fila(df_anual_filtrado,'PEG Ratio','PEGY Ratio')
agregar_fila(df_anual_filtrado,'PE Ratio','PER')
agregar_fila(df_anual_filtrado,'PS Ratio','P/S Ratio')
agregar_razon_financiera(df_anual_filtrado,'  Accounts Receivable','Receivable to Current Assets','Total Current Assets',como_porcentaje=True)
agregar_fila(df_anual_filtrado,'ROA %','ROA%')
agregar_fila(df_anual_filtrado,'ROE %','ROE%')
agregar_razon_financiera(df_anual_filtrado,'Total Assets','ROB','Total Equity',como_porcentaje=False)
agregar_fila(df_anual_filtrado,'ROIC %','ROIC %')
agregar_fila(df_anual_filtrado,'WACC %','WACC %')
agregar_razon_financiera_numerador(df_anual_filtrado,'Test de Acidez','Total Current Liabilities','Total Current Assets','Total Inventories',como_porcentaje=False)
agregar_fila(df_anual_filtrado,'Book Value per Share','Valor en libros')
agregar_fila(df_anual_filtrado,'Piotroski F-Score','Piotrivski F-Score')
agregar_fila(df_anual_filtrado,'Altman Z-Score','Altman Z score')
agregar_fila(df_anual_filtrado,'Beneish M-Score','Beneish M-Score')
agregar_fila(df_anual_filtrado,'Dividends per Share','Dividendo por accion')
agregar_crecimiento_anual('Dividendo por accion','Crecimiento Dividendo por accion')
razones_financieras

,Dec2013,Dec2014,Dec2015,Dec2016,Dec2017,Dec2018,Dec2019,Dec2021,Dec2022,Dec2023
Amortizacion(%),5.67,5.79,5.96,6.56,7.02,6.23,8.30,9.12,7.59,6.91
Beneficio Neto(%),10.39,9.56,10.09,10.10,8.09,8.69,8.63,8.54,9.90,10.00
Beneficio Neto,7701.93,7763.48,9210.729,10140.432,9885.69,11704.347,12383.12,12868.176,17384.903,19486.518
Margen Bruto(%),40.44,40.66,40.69,40.16,39.93,39.42,39.50,37.66,38.76,40.01
Margen Operativo(%),14.59,13.68,16.01,13.35,14.99,15.05,16.08,12.43,14.53,15.20
EPS,5.74,5.78,6.86,7.56,7.37,8.72,9.25,9.56,12.95,14.52
Revenue Per Share,55.21,60.51,68.02,74.83,91.02,100.39,107.16,111.95,130.88,145.14
# de Acciones,1342.196,1342.196,1342.196,1342.196,1342.196,1342.196,1339.323,1346.254,1342.206,1342.196
Ventas,74105.444,81213.589,91292.889,100441.536,122168.279,134739.99,143527.524,150709.751,175663.359,194812.281
Crecimiento Beneficio Neto,0.00,0.80,18.64,10.09,-2.51,18.40,5.80,3.92,35.10,12.09


In [18]:
razones_financieras.loc["Beneish M-Score"] = razones_financieras.loc["Beneish M-Score"].astype(float).abs()
razones_financieras.loc["Gasto financiero"] = razones_financieras.loc["Gasto financiero"].astype(float).abs()

In [19]:
# Convertir los valores a numéricos, convirtiendo los no numéricos a NaN
razones_financieras_numeric = razones_financieras.apply(pd.to_numeric, errors='coerce')

# Ignorar las columnas que contienen "2020" o "2021" para las filas específicas
filas_a_ignorar = ["Crecimiento Beneficio Neto", "Crecimiento Costo de Ventas", "Crecimiento Inventarios", "Crecimiento Ventas", "Crecimiento Dividendo por accion"]

# Crear una copia del DataFrame para modificar
razones_financieras_modificado = razones_financieras_numeric.copy()

# Establecer los valores de las columnas a ignorar en NaN para las filas específicas
for fila in filas_a_ignorar:
    for col in razones_financieras_modificado.columns:
        if '2020' in col or '2021' in col:
            razones_financieras_modificado.loc[fila, col] = np.nan

# Calcular el promedio de cada fila, ignorando los NaN y valores no finitos
promedios = razones_financieras_modificado.apply(lambda row: np.nanmean(row[np.isfinite(row)]), axis=1)

# Redondear los valores a dos decimales
promedios_redondeados = promedios.round(2)

# Convertir los promedios a un DataFrame
promedios_df = promedios_redondeados.to_frame(name='Promedio')
promedios_df

,Promedio
Amortizacion(%),6.92
Beneficio Neto(%),9.40
Beneficio Neto,11852.93
Margen Bruto(%),39.72
Margen Operativo(%),14.59
EPS,8.83
Revenue Per Share,94.51
# de Acciones,1342.32
Ventas,126867.46
Crecimiento Beneficio Neto,10.93


## Dividend Discount Model

$$
P_0 = \frac{D_1}{r - g}
$$

In [20]:
# Cargar datos del stock
data_stock = pd.read_csv(archivo_stock + '.csv')
data_stock['Fecha'] = pd.to_datetime(data_stock['Fecha'], format='%d.%m.%Y')
data_stock['Cierre'] = data_stock['Cierre'].astype(str).str.replace(',', '').astype(float)
data_stock = data_stock.sort_values('Fecha')

# Filtrar los últimos 5 años
ultima_fecha_2023 = data_stock[data_stock['Fecha'].dt.year == 2023]['Fecha'].max()
fecha_actual_stock = ultima_fecha_2023
fecha_inicio_stock = fecha_actual_stock - pd.DateOffset(years=5)
datos_x_anos_stock = data_stock[data_stock['Fecha'] >= fecha_inicio_stock]

# Calcular el crecimiento basado en precios
precio_inicial_stock = datos_x_anos_stock.iloc[0]['Cierre']
precio_final_stock = datos_x_anos_stock.iloc[-1]['Cierre']


n = 5  # número de años
crecimiento_stock = ((precio_final_stock / precio_inicial_stock) ** (1 / n)) - 1
crecimiento_stock_porcentaje = crecimiento_stock * 100

# Sumar los dividendos de los últimos 5 años
ultimas_5_columnas = razones_financieras.columns[-n:]
dividendos_ultimos_5_anios = razones_financieras.loc["Dividendos por Accion", ultimas_5_columnas]
suma_dividendos_ultimos_5_anios = dividendos_ultimos_5_anios.sum()

# Calcular el crecimiento total ajustado por dividendos
precio_final_stock_ajustado = precio_final_stock + suma_dividendos_ultimos_5_anios
crecimiento_stock_irt = ((precio_final_stock_ajustado / precio_inicial_stock) ** (1 / n)) - 1
crecimiento_stock_irt_porcentaje = crecimiento_stock_irt * 100

print(f"5-Year Growth Stock: {crecimiento_stock_porcentaje:.2f}%")
print(f"5-Year Growth Stock IRT: {crecimiento_stock_irt_porcentaje:.2f}%")


5-Year Growth Stock: -6.78%
5-Year Growth Stock IRT: -4.96%


In [21]:
# Seleccionar las últimas 3 columnas del DataFrame
ultimas_3_columnas = razones_financieras.columns[-4:]

# Seleccionar los valores de la fila "Crecimiento Dividendo por accion" para las últimas 3 columnas
crecimiento_dividendo_ultimas_3 = razones_financieras.loc["Crecimiento Dividendo por accion", ultimas_3_columnas]

crecimiento_dividendo_ultimas_3 = crecimiento_dividendo_ultimas_3.loc[~crecimiento_dividendo_ultimas_3.index.str.contains("2021")]

# Calcular el promedio de los valores seleccionados, ignorando NaN y valores no finitos
promedio_crecimiento_dividendo = crecimiento_dividendo_ultimas_3.apply(pd.to_numeric, errors='coerce').mean()

# Redondear el promedio a dos decimales
promedio_crecimiento_dividendo_redondeado = round(promedio_crecimiento_dividendo, 2)

# Mostrar el resultado
g = round(promedio_crecimiento_dividendo_redondeado/100,4)
# Seleccionar la última columna del DataFrame
ultima_columna = razones_financieras.columns[-1]

# Obtener el valor del último dividendo de la fila "Dividendos por Accion"
ultimo_dividendo = razones_financieras.loc["Dividendos por Accion", ultima_columna]


In [22]:
##Calcular R tasa de descuento con CAPM
r = cetes + beta * (crecimiento_ipc - cetes)

##Calcular D1 para conocer el dividendo el proximo año
d1  = ultimo_dividendo * (1 + g)

p0= d1/(r-g)

##Si p0 sale negativo entonces no puede usarse XD
print(p0)

-65.31691975031335


In [23]:
r

0.05604507972857635

## Book Value per Share

In [24]:
total_assets = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Total Assets'].iloc[0, 1:]
total_liabilities = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Total Liabilities'].iloc[0, 1:]
total_shares = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Shares Outstanding (Basic Average)'].iloc[0, 1:]

In [25]:
# Calcular el valor en libros (Book Value) para cada año
book_value = total_assets - total_liabilities

# Calcular el valor en libros por acción (Book Value per Share) para cada año
book_value_per_share = book_value / total_shares

# Crear un DataFrame para almacenar los resultados
book_value_per_share_anual = pd.DataFrame({
    'Book Value': book_value,
    'Book Value per Share': book_value_per_share
}, index=total_assets.index)

book_value_per_share_anual

,Book Value,Book Value per Share
0,,
Dec2013,54827.332,40.848976
Dec2014,62666.421,46.689471
Dec2015,71344.812,53.155286
Dec2016,81335.496,60.598822
Dec2017,90082.378,67.115666
Dec2018,100700.29,75.026516
Dec2019,109074.538,81.440054
Dec2021,119887.519,89.052674
Dec2022,132454.516,98.684193


## Liquidation Value


Utilizable con empresas a la baja, si el precio se encuentra por debajo de este,podria ser una oportunidad

1️⃣ Ratio de Recuperación del Efectivo (
𝑅
𝐸
R 
E
​
 )
(Normalmente 100%)
Se incluyen todas las cuentas que representan efectivo o equivalentes de efectivo, que son altamente líquidos.

Cuentas relevantes:

Cash And Cash Equivalents
Marketable Securities
Cash, Cash Equivalents, Marketable Securities (si está consolidado)
2️⃣ Ratio de Recuperación de Cuentas por Cobrar (
𝑅
𝐶
R 
C
​
 )
(Usualmente entre 70-90%)
Se incluyen todas las cuentas por cobrar, notas por cobrar y otros montos adeudados a la empresa.

Cuentas relevantes:

Accounts Receivable
Notes Receivable
Loans Receivable
Other Current Receivables
Total Receivables (si está consolidado)
3️⃣ Ratio de Recuperación de Inventarios (
𝑅
𝐼
R 
I
​
 )
(Suele ser 50-80%)
Incluye todos los bienes en inventario que la empresa posee, como materia prima, productos en proceso y productos terminados.

Cuentas relevantes:

Inventories, Raw Materials & Components
Inventories, Work In Process
Inventories, Finished Goods
Inventories, Other
Total Inventories (si está consolidado)
4️⃣ Ratio de Recuperación de Propiedades, Planta y Equipos (
𝑅
𝑃
R 
P
​
 )
(Usualmente 60-90%)
Incluye activos fijos tangibles como edificios, maquinaria y terrenos.

Cuentas relevantes:

Land And Improvements
Buildings And Improvements
Machinery, Furniture, Equipment
Construction In Progress
Other Gross PPE
Gross Property, Plant and Equipment
Property, Plant and Equipment (si está consolidado)
⚠️ Acumulated Depreciation → Se resta porque representa la pérdida de valor de los activos.
5️⃣ Ratio de Recuperación de Otros Activos Tangibles (
𝑅
𝑂
R 
O
​
 )
(Depende del tipo de activo, entre 10-80%)
Incluye otros activos físicos que no encajan en las categorías anteriores, como infraestructura, vehículos y mobiliario.

Cuentas relevantes:

Investments And Advances (si contiene activos tangibles)
Other Long Term Assets (si son tangibles)
Otros activos que no sean intangibles o Goodwill
⚠️ No se incluyen:

Intangible Assets (valor de marca, patentes, etc.)
Goodwill (ya que en liquidación normalmente no tiene valor)

In [26]:
ratio_recuperacion_efectivo = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Cash, Cash Equivalents, Marketable Securities'].iloc[0, 1:] * 0.95

ratio_recuperacion_cc = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Total Receivables'].iloc[0, 1:] * 0.75

ratio_recuperacion_inv = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Total Inventories'].iloc[0, 1:] * 0.60

ratio_recuperacion_ppe = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Property, Plant and Equipment'].iloc[0, 1:] * 0.70

ratio_recuperacion_act_tangibles =df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Investments And Advances'].iloc[0, 1:] + df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Other Long Term Assets'].iloc[0, 1:] *.30

total_liabilities = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Total Liabilities'].iloc[0, 1:]


In [27]:
liquidation_value = ratio_recuperacion_efectivo + ratio_recuperacion_cc + ratio_recuperacion_inv + ratio_recuperacion_ppe + ratio_recuperacion_act_tangibles - total_liabilities

liquidation_value_per_share = liquidation_value / total_shares

## EPV

In [28]:
##Obtener el EBIT de promedio de los ultimos 10 años

ultimos_5_anios = razones_financieras.columns[-5:]
ebit_ultimas_5 = razones_financieras.loc["Margen Operativo(%)", ultimos_5_anios]
ebit_promedio=ebit_ultimas_5.apply(pd.to_numeric, errors='coerce').mean()
ebit_promedio_redondeado = round(ebit_promedio, 2) /100
ebit_promedio_redondeado

0.1466

In [29]:
##Obtener las ventas de promedio de los ultimos 10 años

ventas_ultimas_5 = razones_financieras.loc["Ventas", ultimos_5_anios]
ventas_ultimas_5_promedio = ventas_ultimas_5.apply(pd.to_numeric, errors='coerce').mean()
ventas_ultimas_5_promedio_redondeado = round(ventas_ultimas_5_promedio, 2)
ventas_ultimas_5_promedio_redondeado

159890.58

In [30]:
##EL SGYA
SGyA = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Selling, General, & Admin. Expense'].iloc[0, 1:]
# Seleccionar las últimas 5 columnas del DataFrame
ultimas20_columnas = SGyA.index[-20:]
# Obtener los valores de las últimas 5 columnas y multiplicar por 0.75
SGyA_ultimos_20 = SGyA[ultimas20_columnas] * 0.75
SGyA_promedio = round(SGyA_ultimos_20.mean(),2)
# SGyA_promedio = 14361.25

In [31]:
EBIT_normalizado = round((ventas_ultimas_5_promedio_redondeado * ebit_promedio_redondeado) + SGyA_promedio,2)
EBIT_normalizado

34145.32

In [32]:
tax_rate = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Tax Rate %'].iloc[0, 1:]
# Seleccionar las últimas 5 columnas del DataFrame
ultimas_5_columnas = tax_rate.index[-5:]
tax_rate_ultimos_5 = tax_rate[ultimas_5_columnas]
tax_rate_positivos = tax_rate_ultimos_5[tax_rate_ultimos_5 >= 0]
# Calcular el promedio de los valores positivos
promedio_tax_rate = tax_rate_positivos.mean()

# Redondear el promedio a dos decimales
promedio_tax_rate_redondeado = round(promedio_tax_rate, 2)/100
promedio_tax_rate_redondeado

0.25379999999999997

In [33]:
after_tax_normalized_ebit = EBIT_normalizado * (1 - promedio_tax_rate_redondeado )
after_tax_normalized_ebit

25479.237783999997

In [34]:
##Promedio depreciacion
DDA = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Depreciation, Depletion and Amortization'].iloc[0, 1:]
DDA_ultimos_5 = DDA[ultimas_5_columnas]
DDA_promedio = round(DDA_ultimos_5.mean(),2)
Depreciacion_en_exceso= DDA_promedio * 0.5 * promedio_tax_rate_redondeado
DDA_promedio

4749.81

In [35]:
Normalized_earnings = after_tax_normalized_ebit + Depreciacion_en_exceso
Normalized_earnings

26081.988672999996

In [36]:
# Calcular el Maintenance Capex 
revenue = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Revenue'].iloc[0, 1:]
revenue_ultimos_5 = revenue[ultimas_5_columnas]
capex = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Capital Expenditure'].iloc[0, 1:]
capex_ultimos_5 = capex[ultimas_5_columnas].abs()
gppe = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Gross Property, Plant and Equipment'].iloc[0, 1:]
gppe_ultimos_5 = gppe[ultimas_5_columnas]
revenue_cambio = revenue_ultimos_5.diff().fillna(0)
accumulated_depreciation = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == '  Accumulated Depreciation'].iloc[0, 1:]
accumulated_depreciation_ultimos_5 = accumulated_depreciation[ultimas_5_columnas].abs()
net_ppe = gppe_ultimos_5 - accumulated_depreciation_ultimos_5

In [37]:
##maintenance CAPEX

revenue_cambio = revenue_ultimos_5.diff().fillna(0)
net_ppe = gppe_ultimos_5 - accumulated_depreciation_ultimos_5

# 1. Calcular growth_capex solo si hay aumento en revenue
growth_capex = (net_ppe / revenue_ultimos_5) * revenue_cambio
growth_capex = growth_capex.where(revenue_cambio > 0, 0)

# 2. Calcular mantenimiento preliminar
maintenance_capex = capex_ultimos_5 - growth_capex

# 3. Si revenue_cambio fue negativo o el resultado fue negativo, se usa todo el CAPEX
maintenance_capex = maintenance_capex.where(
    (revenue_cambio > 0) & (maintenance_capex >= 0),
    capex_ultimos_5
)

# 4. Promedio
maintenance_capex_promedio = maintenance_capex.mean()

maintenance_capex_promedio

3860.1293954768166

WACC=( 
V
E
​
 ×r 
e
​
 )+( 
V
D
​
 ×r 
d
​
 ×(1−t))
 

In [38]:
promedio_tax_rate_redondeado

0.25379999999999997

In [39]:
##Calcular el WACC ARREGLAR EL WACC, al parecer esta bajo mi resultado
re  = r
precio_de_la_accion = 100.30
E = total_shares * precio_de_la_accion
E = E.iloc[-1]
Deuda_corto_plazo = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Short-Term Debt & Capital Lease Obligation'].iloc[0, 1:]
Deuda_corto_plazo = Deuda_corto_plazo.iloc[-1]
Deuda_largo_plazo = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Long-Term Debt & Capital Lease Obligation'].iloc[0, 1:]
Deuda_largo_plazo = Deuda_largo_plazo.iloc[-1] 
D = Deuda_corto_plazo + Deuda_largo_plazo

market_risk_premium = 0.06 ##Se toma 6% porque historicamente es el rendimiento que ha existido pero tambien podemos calcular expected market return - risk free rate 10yrs

Cost_of_equity = cetes + beta * market_risk_premium
interest_expense = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == '  Interest Expense'].iloc[0, 1:].abs()
interest_expense = interest_expense.iloc[-1]
rd = interest_expense / D

WACC = E / (E + D) * Cost_of_equity + D / (E + D) * rd * (1 - promedio_tax_rate_redondeado)
weigth_equity = E / (E + D)
weigth_debt = D / (E + D)
interest_expense

4067.381

In [40]:
epv_business_operation = (Normalized_earnings - maintenance_capex_promedio) / WACC
cash_and_equivalents = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Cash, Cash Equivalents, Marketable Securities'].iloc[0, 1:]
cash_and_equivalents = cash_and_equivalents.iloc[-1] 
total_shares_ultimo = total_shares.iloc[-1]
cash_and_equivalents

30109.393

In [41]:
# Cálculo del EPV Business Operations
EPV_business_operations = (Normalized_earnings - maintenance_capex_promedio) / WACC
# Ajuste final con efectivo y deuda
EPV_final = (EPV_business_operations + cash_and_equivalents - D) / total_shares_ultimo
EPV_final


126.71952333121742

## NET CURRENT ASSET VALUE

Mas usado para empresas con altos valores de current assets

In [42]:
total_current_assets = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Total Current Assets'].iloc[0, 1:]
total_current_assets = total_current_assets.iloc[-1]
total_liabilities = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Total Liabilities'].iloc[0, 1:]
total_liabilities = total_liabilities.iloc[-1]
minority_interest = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Minority Interest'].iloc[0, 1:]
minority_interest = minority_interest.iloc[-1]
prefered_stock = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Preferred Stock'].iloc[0, 1:]
prefered_stock = prefered_stock.iloc[-1]

In [43]:
net_current_asset_value = total_current_assets - total_liabilities - minority_interest - prefered_stock
net_current_asset_value_per_share = round(net_current_asset_value / total_shares_ultimo,2)
net_current_asset_value_per_share

-0.61

## Tangicle Book Value

In [44]:
total_equity = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Total Stockholders Equity'].iloc[0, 1:]
# total_equity = total_equity.iloc[-1]
prefered_stock = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Preferred Stock'].iloc[0, 1:]
# prefered_stock = prefered_stock.iloc[-1]
intagible_assets = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Intangible Assets'].iloc[0, 1:]
# intagible_assets = intagible_assets.iloc[-1]

tangible_book_value = total_equity - prefered_stock - intagible_assets

tangible_book_value_per_share = tangible_book_value / total_shares_ultimo

tangible_book_value_per_share

0
Dec2013    39.510677
Dec2014    45.145852
Dec2015     53.15137
Dec2016    58.609115
Dec2017    54.823223
Dec2018     62.57317
Dec2019    69.035978
Dec2021    77.297173
Dec2022    86.908455
Dec2023    98.048476
dtype: object

## Projected Free Cash Flow

In [45]:
ebitda = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'EBITDA'].iloc[0, 1:]
free_cash_flow = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Free Cash Flow'].iloc[0, 1:]
ebitda_crecimiento = round(ebitda.pct_change().mean(),3)
free_cash_flow_crecimiento = round(free_cash_flow.pct_change().mean(),3)
revenue = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Revenue'].iloc[0, 1:]
revenue_crecimiento = round(revenue.pct_change().mean(),3)

##Ahora calculado con CAGR
inicio_ebitda = ebitda.iloc[0]
final_ebitda = ebitda.iloc[-1]
n_ebitda = len(ebitda)
cagr_ebitda = round((final_ebitda / inicio_ebitda)**(1/n_ebitda) - 1,3)

inicio_free_cash_flow = free_cash_flow.iloc[0]
final_free_cash_flow = free_cash_flow.iloc[-1]
n_free_cash_flow = len(free_cash_flow)
cagr_free_cash_flow = round((final_free_cash_flow / inicio_free_cash_flow)**(1/n_free_cash_flow) - 1,3)

inicio_revenue = revenue.iloc[0]
final_revenue = revenue.iloc[-1]
n_revenue = len(revenue)
cagr_revenue = round((final_revenue / inicio_revenue)**(1/n_revenue) - 1,3)

In [46]:
##Obtener el valor de crecimiento mas bajo pero que este entre 4 y 11%
variables = {
    'ebitda_crecimiento': ebitda_crecimiento,
    'free_cash_flow_crecimiento': free_cash_flow_crecimiento,
    'revenue_crecimiento': revenue_crecimiento,
    'cagr_ebitda': cagr_ebitda,
    'cagr_free_cash_flow': cagr_free_cash_flow,
    'cagr_revenue': cagr_revenue
}

# Filtrar solo los valores numéricos reales y dentro del rango
variables_filtradas = {
    k: v for k, v in variables.items()
    if isinstance(v, (int, float)) and 0.04 <= v <= 0.11
}

# Determinar el valor mínimo o usar 5% por defecto
if variables_filtradas:
    variable_minima = min(variables_filtradas, key=variables_filtradas.get)
    valor_minimo = variables_filtradas[variable_minima]
else:
    variable_minima = 'default_5%'
    valor_minimo = 0.05


In [47]:
growth_assumption = valor_minimo * 100
growth_multiple = 8.3459 * (1.07 ** (growth_assumption - 4))
growth_multiple

12.609974855979367

In [48]:
free_cash_flow_6anios= free_cash_flow.iloc[-6:]
free_cash_promedio = free_cash_flow_6anios.mean()
free_cash_flow_4trimestres = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Free Cash Flow'].iloc[0, 1:]
free_cash_flow_4trimestres = free_cash_flow_4trimestres.iloc[-4:].mean()
total_stockholders_equity = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Total Stockholders Equity'].iloc[0, 1:]
total_stockholders_equity = total_stockholders_equity.iloc[-1]

fcf_ajustado = (6*free_cash_promedio)+(0.75*free_cash_flow_4trimestres)
fcf_ajustado = fcf_ajustado / 6.75

factor_inflacion = (1 + data_inflacion_mexico_promedio) ** 3

fcf_ajustado_inflacion = fcf_ajustado * factor_inflacion
fcf_ajustado_inflacion

12623.68729908637

In [49]:

intrinscic_value = (growth_multiple * fcf_ajustado_inflacion+ total_stockholders_equity *0.7) / total_shares_ultimo
intrinscic_value

195.3761044074228

## Median PS value

In [50]:
revenue_10_anios = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Revenue'].iloc[0, 1:-1]
diez_anios = revenue_10_anios.index[-11:]
revenue_10_anios = revenue_10_anios[diez_anios]

total_shares_10_anios = total_shares[diez_anios]

ratio_ps = revenue_10_anios / total_shares_10_anios

ratio_ps.index = [int(index[-4:]) for index in ratio_ps.index]

In [51]:
revenue_last_year = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Revenue'].iloc[:, -1]
total_shares_last_year = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Shares Outstanding (Basic Average)'].iloc[:, -1]

revenue_last_value = float(revenue_last_year.values[0])
total_shares_last_value = float(total_shares_last_year.values[0])

ratio_ps_last = revenue_last_value / total_shares_last_value

revenue_last_year

50    194812.281
Name: Dec2023, dtype: object

In [52]:
archivo_stock_10_anios = 'Datos históricos LIVEPOLC1_10anios'

data_stock = pd.read_csv(archivo_stock_10_anios + '.csv')

# Convertir las fechas y ordenar
data_stock['Fecha'] = pd.to_datetime(data_stock['Fecha'], format='%d.%m.%Y')

data_stock = data_stock.sort_values('Fecha')

# Convertir la columna 'Cierre' a tipo numérico, eliminando caracteres no válidos
data_stock['Cierre'] = data_stock['Cierre'].astype(str).str.replace(',', '').astype(float)

# Verificar valores nulos y eliminarlos
data_stock = data_stock.dropna(subset=['Cierre'])
data_stock['Año'] = data_stock['Fecha'].dt.year
promedio_cierre_por_anio = data_stock.groupby('Año')['Cierre'].mean()


ratio_ps_anual = promedio_cierre_por_anio / ratio_ps

mediana_ratio_ps = ratio_ps_anual.median()



In [53]:
Median_ps_value = ratio_ps_last * mediana_ratio_ps

Median_ps_value

227.4798132863562

## Graham Number

In [54]:
eps_without_nri = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'EPS without NRI'].iloc[0, 1:]

tangible_book_value_per_share = tangible_book_value_per_share.astype(float)
eps_without_nri = eps_without_nri.astype(float)

# Aplicar la fórmula: sqrt(22.5 * Tangible Book per Share * EPS without NRI)
graham_number = np.sqrt(22.5 * tangible_book_value_per_share * eps_without_nri)

# Mostrar el resultado
graham_number

0
Dec2013     71.371661
Dec2014     76.650383
Dec2015     97.432543
Dec2016     99.780877
Dec2017    106.169340
Dec2018    122.047299
Dec2019    132.965292
Dec2021    129.011681
Dec2022    159.193296
Dec2023    178.976080
dtype: float64

## Peter Lynch Fair Value

El Valor Justo según Peter Lynch se aplica a empresas en crecimiento. El rango ideal para la tasa de crecimiento es entre 10% y 20% anual.

Peter Lynch considera que el valor justo del P/E (relación precio/utilidad) para una empresa en crecimiento es igual a su tasa de crecimiento, es decir, que el PEG = 1.

Las ganancias utilizadas en este cálculo son las del último año (TTM – trailing twelve months).

Para empresas que no son bancos, la tasa de crecimiento que se usa es el promedio de crecimiento del EBITDA por acción en los últimos 5 años.

Para bancos, se usa el promedio de crecimiento del Valor en Libros por acción en los últimos 5 años.

Si la tasa de crecimiento a 5 años es mayor al 25% anual, se limita a 25%.
Si la tasa de crecimiento a 5 años es menor al 5% anual, no se calcula el Valor Justo de Peter Lynch.

In [55]:
peg_ratio = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'PEG Ratio'].iloc[0, 6:]
ebitda5anios = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'EBITDA'].iloc[0, 6:]
ebitda5anioscrecimiento = round(ebitda5anios.pct_change().mean(),3)*100
ultimo_valor_peg_ratio = peg_ratio.iloc[-1]
ultimo_eps_without_nri = eps_without_nri.iloc[-1]


peter_lynch_value_1 = 1 * ebitda5anioscrecimiento * ultimo_eps_without_nri
peter_lynch_value_2 = ultimo_valor_peg_ratio * ebitda5anioscrecimiento * ultimo_eps_without_nri

peter_lynch_value_min = min(peter_lynch_value_1, peter_lynch_value_2)
peter_lynch_value_min

130.0992

## DCF (FCF BASED)

In [56]:
discount_rate = cetes + market_risk_premium

growth_rates = free_cash_flow.pct_change().dropna()
mediana_growth = growth_rates.median()
weighted_growth = (growth_rates * free_cash_flow.shift(1)).sum() / free_cash_flow.shift(1).sum()

In [57]:
# Lista de valores (puedes reemplazar estos valores con los que ya tienes calculados)
valores = [free_cash_flow_crecimiento, cagr_free_cash_flow, mediana_growth, weighted_growth] 

# Filtrar los valores que están entre 5% y 25%
valores_filtrados = [v for v in valores if 0.05 <= v <= 0.25]

# Aplicar la lógica
if not valores_filtrados:  # Si no hay valores entre 5% y 25%
    resultado = 0.05  # El valor será 5%
else:
    resultado = min(valores_filtrados)  # Escoger el más bajo entre 5% y 25%

# Si todos los valores están por encima del 25%, el resultado será 25%
if all(v > 0.25 for v in valores):
    resultado = 0.25
# Mostrar el resultado
g1 = resultado


In [58]:
y1 = 10
g2 = 0.04
y2 = 10
fcf_per_share = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Free Cash Flow per Share'].iloc[0, 1:]
fcf_per_share = fcf_per_share.iloc[-1]
d =  discount_rate
x = (1+g1)/(1+d)
y = (1+g2)/(1+d)

In [59]:
intrinsic_value_fcf = fcf_per_share * (
    x * (1 - x**10) / (1 - x) +
    x**10 * y * (1 - y**10) / (1 - y)
)

intrinsic_value_fcf

186.62723453474717

## DCF(Earnigs Based)

In [60]:
eps_without_nri_pct_change = round(eps_without_nri.pct_change().mean(),3)

inicio_eps_without_nri = eps_without_nri.iloc[0]
final_eps_without_nri = eps_without_nri.iloc[-1]
n_eps_without_nri = len(eps_without_nri)
cagr_eps_without_nri = round((final_eps_without_nri / inicio_eps_without_nri)**(1/n_eps_without_nri) - 1,3)

growth_rates_eps = eps_without_nri.pct_change().dropna()
mediana_growth_eps = growth_rates_eps.median()

weighted_growth_eps = (growth_rates_eps * eps_without_nri.shift(1)).sum() / eps_without_nri.shift(1).sum()


In [61]:
# Lista de valores (puedes reemplazar estos valores con los que ya tienes calculados)
valores = [eps_without_nri_pct_change, cagr_eps_without_nri, mediana_growth_eps, weighted_growth_eps] 

# Filtrar los valores que están entre 5% y 25%
valores_filtrados = [v for v in valores if 0.05 <= v <= 0.25]

# Aplicar la lógica
if not valores_filtrados:  # Si no hay valores entre 5% y 25%
    resultado = 0.05  # El valor será 5%
else:
    resultado = min(valores_filtrados)  # Escoger el más bajo entre 5% y 25%

# Si todos los valores están por encima del 25%, el resultado será 25%
if all(v > 0.25 for v in valores):
    resultado = 0.25
# Mostrar el resultado
g1 = resultado

In [62]:
y1 = 10
g2 = 0.04
y2 = 10
fcf_per_share = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Free Cash Flow per Share'].iloc[0, 1:]
fcf_per_share = fcf_per_share.iloc[-1]
d =  discount_rate
x = (1+g1)/(1+d)
y = (1+g2)/(1+d)

In [63]:

eps_without_nri_last = eps_without_nri.iloc[-1]

intrinscic_value_eb = eps_without_nri_last * (
    x * (1 - x**10) / (1 - x) +
    x**10 * y * (1 - y**10) / (1 - y)
)
intrinscic_value_eb

175.41062658965876

## Informacion trimestral calculos

## Dividend Discount Model

Solo funciona en acciones que siempre han ofrecido dividendo

In [64]:
dividendos_ultimos_12_trimestres = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Dividends per Share'].iloc[0, 1:]
dividendos_ultimos_12_trimestres = dividendos_ultimos_12_trimestres.iloc[-12:]

# Calcular el crecimiento trimestre a trimestre
dividendos_filtrados = dividendos_ultimos_12_trimestres[dividendos_ultimos_12_trimestres != 0]

crecimiento_dividendo_trim = dividendos_filtrados.pct_change()

# Reemplazar NaN o valores no definidos con 0
crecimiento_dividendo_trim = crecimiento_dividendo_trim.fillna(0)

promedio_crecimiento_dividendo_trim = crecimiento_dividendo_trim.mean()

g_trim = round(promedio_crecimiento_dividendo_trim,4)

ultimo_dividendo_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Dividends per Share'].iloc[0, 1:]
ultimo_dividendo_trim = ultimo_dividendo_trim.iloc[-1]  # Último dividendo

In [65]:
d1_trim = ultimo_dividendo_trim * (1 + g_trim)
p0_trim= d1_trim/(r-g_trim)
#Si salga negativo entonces no puede usarse XD
p0_trim

-9.979895963450481

## Book value per share

In [66]:
total_assets_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Total Assets'].iloc[0, 1:]

total_assets_trim = total_assets_trim.iloc[-4:].mean()
total_liabilities_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Total Liabilities'].iloc[0, 1:]
total_liabilities_trim = total_liabilities_trim.iloc[-4:].mean()
total_shares_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Shares Outstanding (Basic Average)'].iloc[0, 1:]
total_shares_trim = total_shares_trim.iloc[-1]


In [67]:
book_value_trim = total_assets_trim - total_liabilities_trim

# Calcular el valor en libros por acción (Book Value per Share) para cada año
book_value_per_share_trim = book_value_trim / total_shares_trim

book_value_per_share_trim

108.9438117037016

## Liquidation Value

In [68]:
ratio_recuperacion_efectivo_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Cash, Cash Equivalents, Marketable Securities'].iloc[0, 1:] * 0.95

ratio_recuperacion_efectivo_trim = ratio_recuperacion_efectivo_trim.iloc[-4:].mean() 

ratio_recuperacion_cc_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Total Receivables'].iloc[0, 1:] * 0.75

ratio_recuperacion_cc_trim = ratio_recuperacion_cc_trim.iloc[-4:].mean()

ratio_recuperacion_inv_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Total Inventories'].iloc[0, 1:] * 0.60

ratio_recuperacion_inv_trim = ratio_recuperacion_inv_trim.iloc[-4:].mean()

ratio_recuperacion_ppe_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Property, Plant and Equipment'].iloc[0, 1:] * 0.70

ratio_recuperacion_ppe_trim = ratio_recuperacion_ppe_trim.iloc[-4:].mean()

ratio_recuperacion_act_tangibles_trim =df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Investments And Advances'].iloc[0, 1:] + df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Other Long Term Assets'].iloc[0, 1:] *.30

ratio_recuperacion_act_tangibles_trim = ratio_recuperacion_act_tangibles_trim.iloc[-4:].mean()

total_liabilities_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Total Liabilities'].iloc[0, 1:]

total_liabilities_trim = total_liabilities_trim.iloc[-4:].mean()

In [69]:
liquidation_value_trim = ratio_recuperacion_efectivo_trim + ratio_recuperacion_cc_trim + ratio_recuperacion_inv_trim + ratio_recuperacion_ppe_trim + ratio_recuperacion_act_tangibles_trim - total_liabilities_trim

liquidation_value_per_share_trim = liquidation_value_trim / total_shares_trim
liquidation_value_per_share_trim

26.627272989573417

## EPV

In [70]:
margen_op_20_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Operating Margin %'].iloc[0, 1:]/100

margen_op_20_trim = margen_op_20_trim.iloc[-20:]

ebit_promedio_trim=margen_op_20_trim.apply(pd.to_numeric, errors='coerce').mean()

ebit_promedio_redondeado_trim = round(ebit_promedio_trim, 4)

ebit_promedio_redondeado_trim

0.1385

In [71]:
ventas_20_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Revenue'].iloc[0, 1:]
ventas_20_trim  = ventas_20_trim.iloc[-20:]
ventas_20_trim_promedio = ventas_20_trim.apply(pd.to_numeric, errors='coerce').mean()
ventas_20_trim_promedio_redondeado = round(ventas_20_trim_promedio, 4)*4
ventas_20_trim_promedio_redondeado

167002.9864

In [72]:
SGyA_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Selling, General, & Admin. Expense'].iloc[0, 1:]
# Seleccionar las últimas 5 columnas del DataFrame
ultimas20_columnas = SGyA_trim.index[-20:]
# Obtener los valores de las últimas 5 columnas y multiplicar por 0.75
SGyA_ultimos_20_trim = SGyA[ultimas20_columnas] * 0.25
SGyA_promedio_trim = round(SGyA_ultimos_20_trim.mean(),2)*4
SGyA_promedio_trim

14273.8

In [73]:
EBIT_normalizado_trim = round((ventas_20_trim_promedio_redondeado * ebit_promedio_redondeado_trim) + SGyA_promedio_trim,2)
EBIT_normalizado_trim

37403.71

In [74]:
tax_rate_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Tax Rate %'].iloc[0, 1:]
# Seleccionar las últimas 5 columnas del DataFrame
ultimas20_columnas = tax_rate_trim.index[-20:]
tax_rate_ultimos_20 = tax_rate_trim[ultimas20_columnas]
tax_rate_ultimos_20_positivos = tax_rate_ultimos_20[tax_rate_ultimos_20 >= 0]
# Calcular el promedio de los valores positivos 
promedio_tax_rate_trim = tax_rate_ultimos_20_positivos.mean()
# Redondear el promedio a dos decimales
promedio_tax_rate_redondeado_trim = round(promedio_tax_rate_trim, 4)/100
promedio_tax_rate_redondeado_trim

0.24710000000000001

In [75]:
after_tax_normalized_ebit_trim = EBIT_normalizado_trim * (1 - promedio_tax_rate_redondeado_trim )
after_tax_normalized_ebit_trim

28161.253259

In [76]:
DDA_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Depreciation, Depletion and Amortization'].iloc[0, 1:]
# Seleccionar las últimas 5 columnas del DataFrame
ultimas20_columnas = DDA.index[-20:]

DDA_ultimos_20 = DDA[ultimas20_columnas]
DDA_promedio_trim = round(DDA_ultimos_20.mean(),2)*4
Depreciacion_en_exceso_trim= DDA_promedio_trim * 0.5 * promedio_tax_rate_redondeado_trim
Depreciacion_en_exceso_trim

1761.4424660000002

In [77]:
Normalized_earnings_trim = after_tax_normalized_ebit_trim + Depreciacion_en_exceso_trim
Normalized_earnings_trim

29922.695725

In [78]:
revenue_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Revenue'].iloc[0, 1:]
revenue_20_trim = revenue_trim.iloc[-20:]
capex_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Capital Expenditure'].iloc[0, 1:]
capex_20_trim = capex_trim.iloc[-20:].abs()
gppe_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Gross Property, Plant and Equipment'].iloc[0, 1:]
gppe_20_trim = gppe_trim.iloc[-20:]
revenue_cambio_trim = revenue_20_trim.diff().fillna(0)
accumulated_depreciation_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == '  Accumulated Depreciation'].iloc[0, 1:]
accumulated_depreciation_20_trim = accumulated_depreciation_trim.iloc[-20:].abs()
net_ppe_trim = gppe_20_trim - accumulated_depreciation_20_trim
net_ppe_trim

0
Sep2018    46352.924
Dec2018    47115.104
Mar2019    58653.465
Jun2019    59708.868
Sep2019    60926.322
Dec2019    62089.555
Mar2021    62466.337
Jun2021    62591.313
Sep2021    62734.568
Dec2021    64087.176
Mar2022    63795.478
Jun2022    64658.275
Sep2022    65737.774
Dec2022    67906.815
Mar2023    67208.518
Jun2023     68709.42
Sep2023    70145.849
Dec2023    71945.557
Mar2024    72399.087
Jun2024    73304.396
dtype: object

In [79]:
# 1. Calcular growth_capex solo si hay aumento en revenue
growth_capex_trim = (net_ppe_trim / revenue_20_trim) * revenue_cambio_trim
growth_capex_trim = growth_capex_trim.where(revenue_cambio_trim > 0, 0)

# 2. Calcular mantenimiento preliminar
maintenance_capex_trim = capex_20_trim - growth_capex_trim

# 3. Si revenue_cambio fue negativo o el resultado fue negativo, se usa todo el CAPEX
maintenance_capex_trim = maintenance_capex_trim.where(
    (revenue_cambio_trim > 0) & (maintenance_capex_trim >= 0),
    capex_20_trim
)

# 4. Promedio
maintenance_capex_promedio_trim = maintenance_capex_trim.mean()*4

maintenance_capex_promedio_trim

6884.989199999999

In [80]:
re_trim = r
precio_de_la_accion = 97.30
E_trim = total_shares_trim * precio_de_la_accion

Deuda_corto_plazo_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Short-Term Debt & Capital Lease Obligation'].iloc[0, 1:]
Deuda_corto_plazo_trim = Deuda_corto_plazo_trim.iloc[-4:].mean()
Deuda_largo_plazo_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Long-Term Debt & Capital Lease Obligation'].iloc[0, 1:]
Deuda_largo_plazo_trim  = Deuda_largo_plazo_trim.iloc[-4:].mean()

D_trim = Deuda_corto_plazo_trim + Deuda_largo_plazo_trim

market_risk_premium = 0.06 ##Se toma 6% porque historicamente es el rendimiento que ha existido pero tambien podemos calcular expected market return - risk free rate 10yrs


Cost_of_equity = cetes + beta * market_risk_premium
interest_expense_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == '  Interest Expense'].iloc[0, 1:].abs()
interest_expense_trim = interest_expense_trim.iloc[-4:].mean()
rd_trim = interest_expense_trim / D_trim

WACC_trim = E_trim / (E_trim + D_trim) * Cost_of_equity + D_trim / (E_trim + D_trim) * rd_trim * (1 - promedio_tax_rate_redondeado_trim)
weigth_equity_trim = E_trim / (E_trim + D_trim)
weigth_debt_trim = D_trim / (E_trim + D_trim)
WACC_trim


0.10885781249875764

In [81]:
epv_business_operation_trim = (Normalized_earnings_trim - maintenance_capex_promedio_trim) / WACC_trim
cash_and_equivalents_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Cash, Cash Equivalents, Marketable Securities'].iloc[0, 1:]
cash_and_equivalents_trim = cash_and_equivalents_trim.iloc[-4:].mean() 
total_shares_ultimo = total_shares_last_value
cash_and_equivalents_trim

22820.2725

In [82]:
EPV_business_operations_trim = (Normalized_earnings_trim - maintenance_capex_promedio_trim) / WACC_trim
# Ajuste final con efectivo y deuda
EPV_final_trim = (EPV_business_operations_trim + cash_and_equivalents_trim - D_trim) / total_shares_ultimo
EPV_final_trim

143.670619474751

## Net Current Asset Value

In [83]:
total_current_assets_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Total Current Assets'].iloc[0, 1:]
total_current_assets_trim = total_current_assets_trim.iloc[-4:].mean() 

total_liabilities_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Total Liabilities'].iloc[0, 1:]
total_liabilities_trim = total_liabilities_trim.iloc[-4:].mean() 
minority_interest_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Minority Interest'].iloc[0, 1:]
minority_interest_trim = minority_interest_trim.iloc[-4:].mean() 
prefered_stock_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Preferred Stock'].iloc[0, 1:]
prefered_stock_trim = prefered_stock_trim.iloc[-4:].mean() 

In [84]:
net_current_asset_value_trim = total_current_assets_trim - total_liabilities_trim - minority_interest_trim - prefered_stock_trim
net_current_asset_value_per_share_trim = round(net_current_asset_value_trim / total_shares_ultimo,2)
net_current_asset_value_per_share_trim

-2.67

## Tangible book value

In [85]:
total_equity_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Total Stockholders Equity'].iloc[0, 1:]
total_equity_trim = total_equity_trim.iloc[-4:].mean() 
prefered_stock_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Preferred Stock'].iloc[0, 1:]
prefered_stock_trim = prefered_stock_trim.iloc[-4:].mean() 
intagible_assets_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Intangible Assets'].iloc[0, 1:]
intagible_assets_trim = intagible_assets_trim.iloc[-4:].mean() 

tangible_book_value_trim = total_equity_trim - prefered_stock_trim - intagible_assets_trim
tangible_book_value_per_share_trim = tangible_book_value_trim / total_shares_ultimo

tangible_book_value_per_share_trim

97.17981092180278

## Projected Free Cash Flow

In [86]:
ebitda_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'EBITDA'].iloc[0, 1:]
free_cash_flow_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Free Cash Flow'].iloc[0, 1:]
ebitda_crecimiento_trim = round(ebitda_trim.pct_change().mean(),3)
free_cash_flow_crecimiento_trim = round(free_cash_flow_trim.pct_change().mean(),3)
revenue_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Revenue'].iloc[0, 1:]
revenue_crecimiento_trim = round(revenue_trim.pct_change().mean(),3)

##Ahora calculado con CAGR
inicio_ebitda_trim = ebitda_trim.iloc[0]
final_ebitda_trim = ebitda_trim.iloc[-1]
n_ebitda_trim = len(ebitda_trim)
cagr_ebitda_trim = round((final_ebitda_trim / inicio_ebitda_trim)**(1/n_ebitda_trim) - 1,3)

inicio_free_cash_flow_trim = free_cash_flow_trim.iloc[0]
final_free_cash_flow_trim = free_cash_flow_trim.iloc[-1]
n_free_cash_flow_trim = len(free_cash_flow_trim)
cagr_free_cash_flow_trim = (final_free_cash_flow_trim / inicio_free_cash_flow_trim)**(1/n_free_cash_flow_trim) - 1

inicio_revenue_trim = revenue_trim.iloc[0]
final_revenue_trim = revenue_trim.iloc[-1]
n_revenue_trim = len(revenue_trim)
cagr_revenue_trim = (final_revenue_trim / inicio_revenue_trim)**(1/n_revenue_trim) - 1

In [87]:
##Obtener el valor de crecimiento mas bajo pero que este entre 4 y 11%
variables_trim = {
    'ebitda_crecimiento': ebitda_crecimiento_trim,
    'free_cash_flow_crecimiento': free_cash_flow_crecimiento_trim,
    'revenue_crecimiento': revenue_crecimiento_trim,
    'cagr_ebitda': cagr_ebitda_trim,
    'cagr_free_cash_flow': cagr_free_cash_flow_trim,
    'cagr_revenue': cagr_revenue_trim
}

# Filtrar solo los valores numéricos reales y dentro del rango
variables_filtradas_trim = {
    k: v for k, v in variables_trim.items()
    if isinstance(v, (int, float)) and 0.04 <= v <= 0.11
}

# Determinar el valor mínimo o usar 5% por defecto
if variables_filtradas_trim:
    variable_minima_trim = min(variables_filtradas_trim, key=variables_filtradas_trim.get)
    valor_minimo_trim = variables_filtradas_trim[variable_minima_trim]
else:
    variable_minima_trim = 'default_5%'
    valor_minimo_trim = 0.05

In [88]:
growth_assumption_trim = valor_minimo_trim * 100
growth_multiple_trim = 8.3459 * (1.07 ** (growth_assumption_trim - 4))
growth_multiple_trim

8.750680372830846

In [89]:
free_cash_flow_6anios_trim= free_cash_flow_trim.iloc[-24:]
free_cash_promedio_trim = free_cash_flow_6anios_trim.mean()*4
free_cash_flow_4trimestres_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Free Cash Flow'].iloc[0, 1:]
free_cash_flow_4trimestres_trim = free_cash_flow_4trimestres_trim.iloc[-4:].mean()

total_stockholders_equity_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Total Stockholders Equity'].iloc[0, 1:]
total_stockholders_equity_trim = total_stockholders_equity_trim.iloc[-4:].mean() 

fcf_ajustado_trim = (6*free_cash_promedio_trim)+(0.75*free_cash_flow_4trimestres_trim)
fcf_ajustado_trim = fcf_ajustado_trim / 6.75

factor_inflacion_trim = (1 + data_inflacion_mexico_promedio) ** 3

fcf_ajustado_inflacion_trim = fcf_ajustado_trim * factor_inflacion_trim


In [90]:
intrinscic_value_trim = (growth_multiple_trim * fcf_ajustado_inflacion_trim+ total_stockholders_equity_trim *0.7) / total_shares_ultimo
intrinscic_value_trim

158.28337920429638

## Median PS Value

### No se puede hacer con valores trimestrales, debido a que se ocupan media de los ultimos 10 años

## Graham Number

In [91]:
eps_without_nri_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'EPS without NRI'].iloc[0, -4:].mean()*4

graham_number_trim = np.sqrt(22.5 * tangible_book_value_per_share_trim * eps_without_nri_trim)

graham_number_trim

186.70273773728132

## Peter Lynch Fair Value

In [92]:
ultimo_peg_ratio_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'PEG Ratio'].iloc[0,-1:]

# Tomar últimos 20 trimestres de EBITDA
ebitda5anios_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'EBITDA'].iloc[0, -20:]

# Quitar trimestres de 2020
cols_validas = [col for col in ebitda5anios_trim.index if not col.endswith('2020')]
ebitda5anios_trim = ebitda5anios_trim[cols_validas]

# Calcular crecimiento trimestral promedio
crecimiento_trim = ebitda5anios_trim.pct_change().mean()

crecimiento_trim_limited = min(crecimiento_trim, 0.25) *100

peg_ratio_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'PEG Ratio'].iloc[0, 6:]
ultimo_valor_peg_ratio_trim = peg_ratio_trim.iloc[-1]

peter_lynch_value_1_trim = 1 * crecimiento_trim_limited * eps_without_nri_trim
peter_lynch_value_2_trim = ultimo_valor_peg_ratio_trim * crecimiento_trim_limited * eps_without_nri_trim

peter_lynch_value_min_trim = min(peter_lynch_value_1_trim, peter_lynch_value_2_trim)

peter_lynch_value_min_trim


183.333

## DCF (FCF BASED)

In [93]:
discount_rate

0.14350000000000002

In [94]:
growth_rates_trim = free_cash_flow_trim.pct_change().dropna()
mediana_growth_trim = growth_rates_trim.median()
weighted_growth_trim = (growth_rates_trim * free_cash_flow_trim.shift(1)).sum() / free_cash_flow_trim.shift(1).sum()
weighted_growth_trim

0.06529767330517841

In [95]:
# Lista de valores (puedes reemplazar estos valores con los que ya tienes calculados)
valores_trim = [free_cash_flow_crecimiento_trim, cagr_free_cash_flow_trim, mediana_growth_trim, weighted_growth_trim] 

valores_trim = [float(v.real) if isinstance(v, complex) else float(v) for v in valores_trim]

# Filtrar los valores que están entre 5% y 25%
valores_filtrados_trim = [v for v in valores_trim if 0.05 <= v <= 0.25]

# Aplicar la lógica
if not valores_filtrados_trim:  # Si no hay valores entre 5% y 25%
    resultado_trim = 0.05  # El valor será 5%
else:
    resultado_trim = min(valores_filtrados_trim)  # Escoger el más bajo entre 5% y 25%

# Si todos los valores están por encima del 25%, el resultado será 25%
if all(v > 0.25 for v in valores_trim):
    resultado_trim = 0.25
# Mostrar el resultado
g1_trim = resultado_trim
g1_trim

0.06529767330517841

In [96]:
fcf_per_share_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Free Cash Flow per Share'].iloc[0, -4:].mean()*4

d_trim =  discount_rate
x_trim = (1+g1_trim)/(1+d_trim)
y_trim = (1+g2)/(1+d_trim)

In [97]:
intrinsic_value_trim_fcf = fcf_per_share_trim * (
    x_trim * (1 - x_trim**10) / (1 - x_trim) +
    x_trim**10 * y_trim * (1 - y_trim**10) / (1 - y_trim)
)

intrinsic_value_trim_fcf

122.60782608832021

## DCF (Earnings Based)

In [98]:
eps_without_nri_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'EPS without NRI'].iloc[0, 1:]
eps_without_nri_trim = eps_without_nri_trim[~eps_without_nri_trim.index.str.contains('2020')]

eps_without_nri_pct_change_trim = round(eps_without_nri_trim.pct_change().mean(),3)

inicio_eps_without_nri_trim = eps_without_nri_trim.iloc[0]
final_eps_without_nri_trim = eps_without_nri_trim.iloc[-1]
n_eps_without_nri_trim = len(eps_without_nri_trim)
cagr_eps_without_nri_trim = round((final_eps_without_nri_trim / inicio_eps_without_nri_trim)**(1/n_eps_without_nri_trim) - 1,3)

growth_rates_eps_trim = eps_without_nri_trim.pct_change().dropna()
mediana_growth_eps_trim = growth_rates_eps_trim.median()

weighted_growth_eps_trim = (growth_rates_eps_trim * eps_without_nri_trim.shift(1)).sum() / eps_without_nri_trim.shift(1).sum()

In [99]:
# Lista de valores (puedes reemplazar estos valores con los que ya tienes calculados)
valores_trim = [eps_without_nri_pct_change_trim, cagr_eps_without_nri_trim, mediana_growth_eps_trim, weighted_growth_eps_trim] 

# Filtrar los valores que están entre 5% y 25%
valores_filtrados_trim = [v for v in valores_trim if 0.05 <= v <= 0.25]

# Aplicar la lógica
if not valores_filtrados_trim:  # Si no hay valores entre 5% y 25%
    resultado_trim = 0.05  # El valor será 5%
else:
    resultado_trim = min(valores_filtrados_trim)  # Escoger el más bajo entre 5% y 25%

# Si todos los valores están por encima del 25%, el resultado será 25%
if all(v > 0.25 for v in valores_trim):
    resultado_trim = 0.25
# Mostrar el resultado
g1_trim = resultado_trim
g1_trim

0.05

In [100]:
y1_trim = 10
g2_trim = 0.04
y2_trim = 10
eps_without_nri_last_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'EPS without NRI'].iloc[0, -4:].mean()*4

d_trim =  discount_rate
x_trim = (1+g1_trim)/(1+d_trim)
y_trim = (1+g2_trim)/(1+d_trim)


In [101]:
intrinscic_value_eb_trim = eps_without_nri_last_trim * (
    x_trim * (1 - x_trim**10) / (1 - x_trim) +
    x_trim**10 * y_trim * (1 - y_trim**10) / (1 - y_trim)
)
intrinscic_value_eb_trim

144.56818014544154

## Tabla con valores de acciones

In [102]:
p0
book_value_per_share_anual  ## ya viene todos los años
liquidation_value_per_share ## ya viene todos los años
EPV_final
net_current_asset_value_per_share
tangible_book_value_per_share ## ya viene todos los años
intrinscic_value
Median_ps_value
graham_number  ## ya viene todos los años
peter_lynch_value_1
peter_lynch_value_2
intrinsic_value_fcf
intrinscic_value_eb

###Valore de los trimestres
p0_trim
book_value_per_share_trim
liquidation_value_per_share_trim
EPV_final_trim
net_current_asset_value_per_share_trim
tangible_book_value_per_share_trim
intrinscic_value_trim
graham_number_trim
peter_lynch_value_1_trim
peter_lynch_value_2_trim
intrinsic_value_trim_fcf
intrinscic_value_eb_trim

144.56818014544154